In [59]:
import pandas as pd
from textblob import TextBlob
%load_ext autoreload
%autoreload 2
import utils as ut


import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
df_reviews = pd.read_csv('data/csv/ureviews_clean.csv', encoding='utf-8')
df_games = pd.read_csv('data/csv/games_clean.csv', encoding='utf-8')
df_items = pd.read_csv('data/csv/uitems_clean.csv', encoding='utf-8')

Let's create a new column called 'sentiment_analysis' replacing the 'reviews_review' column.
We will carry out a sentiment analysis of the comments in the 'reviews' column, classifying them as follows:

0 if bad,
1 if it is neutral or without review
2 if positive.

In [61]:
# Apply sentiment analysis feature
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: TextBlob(str(x)).sentiment.polarity if pd.notna(x) and x != 'No data' else None)

# Define a function to convert the sentiment analysis to the scale (0, 1, 2)
def map_sentiment(score):
    if score is None:
        return 1  # If the review is absent or 'No data', take the value of 1
    elif score < 0:
        return 0  # bad
    elif score == 0:
        return 1  # neutral
    else:
        return 2  # positive

# Apply the mapping function to the 'sentiment_analysis' column
df_reviews['sentiment_analysis'] = df_reviews['sentiment_analysis'].apply(map_sentiment)

# PlayTime Genre
We will create an auxiliary dataframe that shows us genre, year, hours played per genre.

In [62]:
playtime_item = df_items[['playtime_forever', 'item_id']]
playtime_item

,playtime_forever,item_id
0,6,10
1,0,20
2,7,30
3,0,40
4,0,50
...,...,...
5094100,0,346330
5094101,0,373330
5094102,3,388490
5094103,4,521570


In [63]:
df_genre_year=df_games[['item_id', 'genres', 'release_year']]

In [64]:
df_genre_year.head()

,item_id,genres,release_year
0,761140,Action,2018
1,761140,Casual,2018
2,761140,Indie,2018
3,761140,Simulation,2018
4,761140,Strategy,2018


In [65]:
df_playtime_genre=df_genre_year.merge(playtime_item, on='item_id')
df_playtime_genre

,item_id,genres,release_year,playtime_forever
0,282010,Action,1997,5
1,282010,Action,1997,0
2,282010,Action,1997,0
3,282010,Action,1997,0
4,282010,Action,1997,13
...,...,...,...,...
11109752,80,Action,2004,0
11109753,80,Action,2004,0
11109754,80,Action,2004,0
11109755,80,Action,2004,9


In [66]:
playtime_genre = df_playtime_genre.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()
playtime_genre

,genres,release_year,playtime_forever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18335
...,...,...,...
701,Werewolves,no data,3128
702,Zombies,2015,2728
703,Zombies,no data,78741
704,e-sports,2014,448600


# User for genre
We will create an auxiliary dataframe that shows us genre, year, user, hours played by user per year in each gender.

In [67]:
user_item = df_items[['playtime_forever', 'user_id', 'item_id']]
user_item.head()


,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50


In [68]:
user_for_genre=df_genre_year.merge(user_item, on='item_id')
user_for_genre

,item_id,genres,release_year,playtime_forever,user_id
0,282010,Action,1997,5,UTNerd24
1,282010,Action,1997,0,I_DID_911_JUST_SAYING
2,282010,Action,1997,0,76561197962104795
3,282010,Action,1997,0,r3ap3r78
4,282010,Action,1997,13,saint556
...,...,...,...,...,...
11109752,80,Action,2004,0,76561198273508956
11109753,80,Action,2004,0,76561198282090798
11109754,80,Action,2004,0,943525
11109755,80,Action,2004,9,76561198283312749


In [69]:
user_for_genre = user_for_genre.groupby(['genres', 'user_id', 'release_year'])['playtime_forever'].sum().reset_index()
user_for_genre

,genres,user_id,release_year,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363
...,...,...,...,...
4376355,e-sports,zuilde,2014,7
4376356,e-sports,zwanzigdrei,2014,512
4376357,e-sports,zyxwvutsrqponm,2014,0
4376358,e-sports,zzeee,2014,1


In [70]:
random_rows = 3000000
user_for_genre = user_for_genre.sample(n=random_rows, random_state=42)
user_for_genre

,genres,user_id,release_year,playtime_forever
1609014,Flight,76561198038768759,no data,0
4192890,Strategy,vinzolvar,no data,530
1819067,Free to Play,DylanVidzy,2015,32
3967617,Strategy,76561198068378615,2012,333
460231,Action,Lethalmirage,2005,197
...,...,...,...,...
3579664,Simulation,JrRocket000,2011,894
2113991,Indie,76561198050912891,2005,6
786660,Adventure,76561198044803324,2015,285
259019,Action,76561198071819981,2013,2892


In [71]:
user_for_genre = user_for_genre.groupby(['genres', 'user_id', 'release_year'])['playtime_forever'].sum().reset_index()
user_for_genre

,genres,user_id,release_year,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2014,3827
...,...,...,...,...
2999995,e-sports,zerak_kyria,2014,55
2999996,e-sports,zgeddon,2014,143
2999997,e-sports,zuilde,2014,7
2999998,e-sports,zwanzigdrei,2014,512


# User Recommend

In [72]:
it_recommend = df_games[['item_id', 'item_name']]
it_recommend

,item_id,item_name
0,761140,Lost Summoner Kitty
1,761140,Lost Summoner Kitty
2,761140,Lost Summoner Kitty
3,761140,Lost Summoner Kitty
4,761140,Lost Summoner Kitty
...,...,...
84206,681550,Maze Run VR
84207,681550,Maze Run VR
84208,681550,Maze Run VR
84209,681550,Maze Run VR


In [73]:
u_recommend = df_reviews[['item_id', 'posted', 'recommend', 'sentiment_analysis']]
u_recommend

,item_id,posted,recommend,sentiment_analysis
0,1250,2011,True,2
1,22200,2011,True,2
2,43110,2011,True,2
3,251610,2014,True,2
4,227300,2013,True,0
...,...,...,...,...
48493,730,2015,True,1
48494,253980,2015,True,2
48495,730,2015,True,0
48496,730,2015,True,2


In [74]:
user_recommend = it_recommend.merge(u_recommend, on='item_id')
user_recommend


,item_id,item_name,posted,recommend,sentiment_analysis
0,70,Half-Life,2015,True,0
1,70,Half-Life,2011,True,0
2,70,Half-Life,2014,True,0
3,70,Half-Life,2013,True,2
4,70,Half-Life,2013,True,0
...,...,...,...,...,...
151618,80,Counter-Strike: Condition Zero,2014,False,1
151619,80,Counter-Strike: Condition Zero,2013,True,2
151620,80,Counter-Strike: Condition Zero,2015,True,2
151621,80,Counter-Strike: Condition Zero,2014,True,1


# Sentiment analysis

Now, we going to create an auxiliar dataframe where we can see sentiment analisys per year

In [75]:
release_year = df_games[['item_id', 'release_year']]
release_year = release_year[release_year['release_year'] != 'no data']
release_year

,item_id,release_year
0,761140,2018
1,761140,2018
2,761140,2018
3,761140,2018
4,761140,2018
...,...,...
84200,610660,2018
84201,610660,2018
84202,610660,2018
84203,658870,2017


In [76]:
sentiment = df_reviews[['item_id', 'sentiment_analysis']]
sentiment

,item_id,sentiment_analysis
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,0
...,...,...
48493,730,1
48494,253980,2
48495,730,0
48496,730,2


In [77]:
sentiment_year = release_year.merge(sentiment, on='item_id')
sentiment_year = sentiment_year[['release_year', 'sentiment_analysis']]
sentiment_year

,release_year,sentiment_analysis
0,1998,0
1,1998,0
2,1998,0
3,1998,2
4,1998,0
...,...,...
103431,2004,1
103432,2004,2
103433,2004,2
103434,2004,1


# FILES TO FUNCTIONS

In [78]:
df_reviews

,posted,item_id,recommend,review,user_id,sentiment_analysis
0,2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,2011,22200,True,It's unique and worth a playthrough.,76561197970982479,2
2,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,2014,251610,True,I know what you think when you see this title ...,js41637,2
4,2013,227300,True,For a simple (it's actually not all that simpl...,js41637,0
...,...,...,...,...,...,...
48493,2015,730,True,its FUNNNNNNNN,wayfeng,1
48494,2015,253980,True,Awesome fantasy game if you don't mind the gra...,76561198251004808,2
48495,2015,730,True,Prettyy Mad Game,72947282842,0
48496,2015,730,True,AMAZING GAME 10/10,ApxLGhost,2


In [79]:
dfs = [playtime_genre, user_for_genre, user_recommend, sentiment_year, df_reviews]
# Nombres correspondientes a cada DataFrame
names = ['playtime_genre', 'user_for_genre', 'user_recommend','sentiment_year', 'reviews_clean']

ut.save_to_csv(dfs, names)

DataFrame 'playtime_genre' saved as 'data/csv/playtime_genre.csv'
DataFrame 'user_for_genre' saved as 'data/csv/user_for_genre.csv'
DataFrame 'user_recommend' saved as 'data/csv/user_recommend.csv'
DataFrame 'sentiment_year' saved as 'data/csv/sentiment_year.csv'
DataFrame 'reviews_clean' saved as 'data/csv/reviews_clean.csv'


In [80]:
ut.save_to_pq(dfs, names)

DataFrame 'playtime_genre' save as 'data/playtime_genre.parquet'
DataFrame 'user_for_genre' save as 'data/user_for_genre.parquet'
DataFrame 'user_recommend' save as 'data/user_recommend.parquet'
DataFrame 'sentiment_year' save as 'data/sentiment_year.parquet'
DataFrame 'reviews_clean' save as 'data/reviews_clean.parquet'
